In [1]:
import scipy
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal
import osascript
from gtts import gTTS 
import os 
import pyaudio
import wave
import keyboard  # using module keyboard
import soundfile as sf
import math
import pyloudnorm as pyln
from sys import byteorder
from array import array
from struct import pack

# Extract data
for this part we extract the wave file as well as the meta data. the data file has two channels that we are going to clean up before we use. 

In [3]:
fs, data = wavfile.read('./LDC2002S28.wav')
number_of_samples = data.shape[0]
meta_data = open(r"LDC2002S28-txt.txt")
meta_data = pd.read_csv("LDC2002S28-txt.txt", sep="A:", header=None, engine='python')
meta_data.columns = ["sound limits","description"]

FileNotFoundError: [Errno 2] No such file or directory: 'LDC2002S28-txt.txt'

In [ ]:
data = np.average(data, axis = 1)

In [ ]:
plt.plot(data)

# High pass filter
used a high pass filter to clean the noise from the dc signal setting the lowest frequeency possible to be 10 hz

In [ ]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
data = butter_highpass_filter(data, 10, fs)
plt.plot(data)

# divide the data

In [ ]:
voice_time_limits = meta_data["sound limits"]

voice_time_limits = [i.split(" ")[0:2] for i in voice_time_limits]
voice_time_limits = np.array(voice_time_limits)
voice_time_limits = voice_time_limits.astype(np.float)


In [ ]:
data_number = 0
divided_data = []
for i in voice_time_limits:
    startingpoint = int(i[0]*fs)
    endingpoint = int(i[1]*fs)
    divided_data.append(data[startingpoint:endingpoint])
    data_number +=1
    

In [ ]:
plt.plot(divided_data[1])

In [ ]:
np.save("segmented_voice_data",divided_data)

# Accepting input
Defining functions for accepting and preprocessing input 

In [ ]:
def input(voice_data):
    fs, data = wavfile.read(voice_data)
    number_of_samples = data.shape[0]
    data = np.average(data, axis = 1)
    data = butter_highpass_filter(data, 10, fs)
    plt.plot(data)

In [7]:
THRESHOLD = 600
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 44100

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)
            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 60:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    plt.figure(6)
    plt.plot(data)
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()
def reply(user_volume):
    speaker_volume = int(osascript.osascript('output volume of (get volume settings)')[1])
    print("speaker volume is " + str(speaker_volume))
    print("user volume is "+str(user_volume))
    if(speaker_volume < user_volume):
        new_volume = speaker_volume + (((speaker_volume - user_volume)**2) /100)
    else:
        new_volume = speaker_volume - (((speaker_volume - user_volume)**2) /100)
    print("speaker new volume is "+str(new_volume))
    osascript.osascript("set volume output volume "+str(new_volume)) 
    thing_to_say = 'hello your volume is at'+str(int(new_volume))
    print(thing_to_say)
    os.system("say "+thing_to_say)
    
def voice_input(voice_data):
    fs, data = wavfile.read(voice_data)
    plt.plot(data)
    number_of_samples = data.shape
    plt.plot(data)
    print(data)
    data = butter_highpass_filter(data, 10, fs)
    return data, fs

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    print(b,a,data)
    y = signal.filtfilt(b, a, data)
    return y

def remove_silence_from(amplitudes, threshold):
    silenced = []
    for x in amplitudes:
        if x >= threshold:
            silenced.append(x)
    return silenced


In [56]:
def trim2(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if abs(i)>50.0:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

In [8]:
def getting_loudness3(data, rate):
    AVG_STEP = 500
    PERCENTILE= 70
    
    plt.figure(0)
    plt.plot(data)
    
    # Average data to step size
    avg_data = np.percentile(data[:(len(data)//AVG_STEP)*AVG_STEP].reshape(-1,AVG_STEP),
                             PERCENTILE, axis=1)
    
    #apply hilbert envelope
    hilbert_transformed = np.abs(scipy.signal.hilbert(avg_data))

    #Smooth out hilbert 
    b, a = scipy.signal.ellip(4, 0.01, 120, 0.125)
    fgust = scipy.signal.filtfilt(b, a, hilbert_transformed, method="gust")
    plt.figure(1)
    plt.plot(avg_data)
    plt.plot(fgust)

    #remove silence 
    avg = np.mean(fgust)
    silenced = remove_silence_from(fgust, avg)
    plt.figure(2)
    plt.plot(silenced)
    data = [float(i) for i in silenced]
    data = np.array(data)

    
    rms = math.sqrt((np.sum(np.square(data))/len(data)))
    # peak normalize audio to -1 dB
    peak_normalized_audio = pyln.normalize.peak(data, -1)
    # measure the loudness first 
    meter = pyln.Meter(rate/500) # create BS.1770 meter
    loudness = meter.integrated_loudness(data)
    print("lds: "+ str(loudness))
    loudness normalize audio to -12 dB LUFS
    loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -12.0)
    plt.figure(4)
    plt.plot(loudness_normalized_audio)
    print("lds mean nrm: "+ str(np.mean(loudness_normalized_audio)))
    print("lds median nrm: "+ str(np.median(loudness_normalized_audio)))
    top_ten = int(len(loudness_normalized_audio)/10)
    print("lds max nrm: "+ str(np.mean(np.sort(loudness_normalized_audio)[-2*top_ten:-top_ten])))
    return the exponential regression
    return 1.619772149*1.128504719**loudness


In [9]:
getting_loudness3(data, rate)

NameError: name 'rate' is not defined

In [11]:
if __name__ == '__main__':
    print("please speak a word into the microphone")
    record_to_file('demo.wav')
    print("done - result written to demo.wav")
    data, rate = voice_input('demo.wav')
    new_volume = getting_loudness3(data, rate)
    reply(int(new_volume))

please speak a word into the microphone


NameError: name 'array' is not defined

In [10]:
getting_loudness3(data, rate)

NameError: name 'rate' is not defined